# Aggregate final results

In [2]:
import glob
import pandas as pd


modality_results = {
    modality : pd.concat(map(pd.read_csv, glob.glob(f"csv/{modality}/*multi_*.csv")))
    for modality in ["v", "a", "av"]
}

In [8]:
modality_results["v"].head()

,video_id,suggestions,response
0,E0pr-jw5Nrg_000129.mp4,[],"ambulance siren: No, there is no ambulance sir..."
1,DLncH1MUazE_000005.mp4,[],"ambulance siren: No, there is no ambulance sir..."
2,K0Y8oGYR7lw_000010.mp4,[],"ambulance siren: No, there is no ambulance sir..."
3,eozwlCm0QIo_000030.mp4,[],"ambulance siren: No, there is no ambulance sir..."
4,dG7RFy7mAyY_000030.mp4,[],"ambulance siren: No, there is no ambulance sir..."


In [20]:
import ast 
from collections import defaultdict
from tqdm.auto import tqdm

test_df = pd.read_csv("../../data/test.csv", names=["video_id", "label"])
modality_data = defaultdict(list)

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    video_id = row['video_id']
    labels = row['label']
    
    if (
        ("av" not in modality_results or video_id not in modality_results["av"]["video_id"].values)
        and ("v" not in modality_results or video_id not in modality_results["v"]["video_id"].values)
        and ("a" not in modality_results or video_id not in modality_results["a"]["video_id"].values)
    ):
        continue
    
    for modality in ["av", "a", "v"]:
        
        
        if modality not in modality_results:
            for i in range(1, 11):
                modality_data[f"{modality}_top_{i}"].append("")
                
            continue
        
        modality_df = modality_results[modality]
        modality_prediction = modality_df[modality_df['video_id'] == video_id]["suggestions"]
        if modality_prediction.empty:
            modality_prediction = []
        else:
            modality_prediction = ast.literal_eval(modality_prediction.tolist()[0])
        
        for i in range(1, 11):
            modality_data[f"{modality}_top_{i}"].append(modality_prediction[i] if i < len(modality_prediction) else "")
    
    modality_data["label"].append(labels)
    modality_data["video_id"].append(video_id.split(".")[0])

  0%|          | 0/15446 [00:00<?, ?it/s]

100%|██████████| 15446/15446 [00:25<00:00, 603.70it/s]


In [21]:
pandagpt_resutls = pd.DataFrame(modality_data)
pandagpt_resutls.to_csv("../../data/model-predictions/pandagpt/test_predictions.csv", index=False)